In [1]:
from bse_dsdm.preprocessing.feature_scaling import *
from bse_dsdm.preprocessing.outlier_detection import *
from bse_dsdm.preprocessing.value_encoder import *
from bse_dsdm.preprocessing.value_imputer import *
from bse_dsdm.preprocessing.load_data import *
from bse_dsdm.preprocessing.exploratory_analysis import *
from bse_dsdm.accuracy_testing.cross_validation import *
import pandas as pd
from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import GridSearchCV

LOAD DATA

In [2]:
data=Data()
data.load_data("../train.csv")


In [3]:
data.data=remove_columns_with_na(data.data,80)


Preprocessing

In [4]:
categorical_columns_na=columns_with_na(data.data)[0]
print(categorical_columns_na)
numerical_columns_na=columns_with_na(data.data)[1]
print(numerical_columns_na)


['club_joined', 'player_traits']
['value_eur', 'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']


In [5]:
data.data=impute_values(data.data,numerical_columns_na, impute_type='mean')

In [6]:
data.data=impute_categorical(data.data,categorical_columns_na)

In [7]:
mapping={'work_rate': {'Low/Low': 1, 'Low/Medium': 2, 'Low/High': 3, 'Medium/Low': 4, 'Medium/Medium': 5, 'Medium/High': 6, 'High/Low': 7, 'High/Medium': 8, 'High/High': 9}}

data.data = encode_ordinal_columns(data.data, columns=['work_rate'], mapping=mapping)

In [8]:
data.data['nationality_name'] = target_encode_column(data.data, 'nationality_name', 'height_cm', compute_type='mean')

In [9]:
data.data=encode_categorical_columns(data.data,column='preferred_foot')

c:\Users\julia\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
data.data = convert_to_age(data.data, 'birthday_date',2021)

In [11]:
data.data=extract_string(data.data,'body_type')

In [12]:
data.split_data()


In [13]:
feature_columns=['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle']
target_column=['position']
X=data.data[feature_columns]
y=data.data['position']


In [15]:

model = RandomForestModel(feature_columns, target_column, data.train_data, data.test_data)
model.train()
predictions = model.predict()
print(f"predictions:{predictions}")

accuracy = model.get_accuracy(predictions, data.test_data[target_column])
print(f"Accuracy: {accuracy}")


c:\Users\julia\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


predictions:['LCM' 'RCB' 'LCB' ... 'RCM' 'ST' 'GK']
Accuracy: 0.3836113125453227


In [16]:

model = RandomForestModel(feature_columns, target_column, data.train_data, data.test_data)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

In [17]:

model = RandomForestModel(feature_columns, target_column, data.train_data, data.test_data)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}


best_model = perform_grid_search(model, param_grid)


Fitting 5 folds for each of 81 candidates, totalling 405 fits


c:\Users\julia\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Best Accuracy: 0.3839735503961528
Test Accuracy with Best Hyperparameters: 0.38578680203045684


In [18]:
f1= evaluate_best_model(best_model, model.test_data[model.feature_columns], model.test_data[model.target_column])

F1_score: 0.3319270978573035
